<a href="https://colab.research.google.com/github/wilzamguerrero/SDZ/blob/main/SDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;96mDrive Connected ✓')



#import os
#from IPython.display import clear_output

#save_models_dir = "/content/sample_data"#@param {type:"string"}
#os.chdir(save_models_dir)
#url = "" #@param {type:"string"}
#!wget --no-check-certificate --content-disposition $url

#clear_output()
#print('[1;96mModel Save ✓')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZanim2.gif?raw=true' height="50" alt="netdata"/></center> <a href="https://wilzamguerrero.ml/3c030fffd57c4e3bb4dd62e3971cb6a9"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/how_to_use5.png?raw=true' height="42" alt="netdata"/></center></font></a> <a href="https://zero01101.github.io/openOutpaint/"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/open_out_paint.png?raw=true' height="42" alt="netdata"/></center></font></a> <a href="https://www.painthua.com/"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/paint_hua.png?raw=true' height="42" alt="netdata"/></center></font></a> </a> <a href="https://github.com/ddPn08/kohya-sd-scripts-webui"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/kohya2.png?raw=true' height="42" alt="netdata"/></center></font></a> </a> <a href="https://github.com/Linaqruf/kohya-trainer"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/kohya2.png?raw=true' height="42" alt="netdata"/></center></font></a> </a> <a href="https://zhuyu1997.github.io/open-pose-editor/"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/save.png?raw=true' height="42" alt="netdata"/></center></font></a> </center></font></a>
import os
import shutil
import urllib.request
#from IPython.utils import capture
from IPython.utils.capture import capture_output
from IPython.display import clear_output
import sys
import fileinput
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact, interactive, fixed
from ipywidgets import Layout, SelectMultiple
from IPython.core.display import HTML

zzz_install = "/content/sample_data"
zzz_install_widget = widgets.Text(value=zzz_install, description="Path install")
zzz_Version = widgets.Dropdown(options=["V1.5", "V2.1"], description="Version")
Download_Base_Model = widgets.Text(value="", description="Link Model")
zzz = "zzz"
zzz_widget = widgets.Text(value=zzz, description="zzz")
xyz = (zzz_install+'/'+'zzz')
xyz_widget = widgets.Text(value=xyz, description="xyz")

model_initp = ""
model_init = widgets.Text(value=model_initp, description="Model_init")
models_dirp = ""
models_dir = widgets.Text(value=models_dirp, description="Models_dir")

zzz_Argsp = "--medvram-sdxl"
zzz_Args = widgets.Text(value=zzz_Argsp, description="Args")
zzz_Training = widgets.Dropdown(options=["", "hyako", "dramb"], description="Training")

ctrll5 = widgets.Checkbox(value=False, description="ctrll5")
ctrlxl = widgets.Checkbox(value=False, description="ctrlxl")

mosta = 'models/S t a b l e-d i f f u s i o n'
mostaz = mosta.replace(' ', '')

zzz_Extensions = widgets.Checkbox(value=False, description="Extensions")
zzz_Upscaler = widgets.Checkbox(value=False, description="Upscalers")
zzz_Continue = widgets.Checkbox(value=False, description="CONTINUE / / / ")

zzzconfigV2="--config v2-inference-v.yaml"
zshare = "--share --api --xformers --theme=dark --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae --cors-allow-origins=https://www.painthua.com"

os.chdir(zzz_install_widget.value)
boton_runzzz = widgets.Button(description="RUN > > > S.D.Z")
boton_runzzz.style.button_color = '#1e1e1e'
boton_runzzz.style.font_weight = 'bold'
boton_runzzz.style.color = 'red'


left_box = widgets.VBox([ctrll5])
right_box = widgets.VBox([ctrlxl])
group_box = widgets.HBox([left_box, right_box],layout=widgets.Layout(height='45px', width='100%'))

left_box.layout.max_width = '150px'
right_box.layout.max_width = '170px'
left_box.layout.overflow = 'hidden'
right_box.layout.overflow = 'hidden'

widgets_box = widgets.VBox([zzz_install_widget, Download_Base_Model],layout=widgets.Layout(height='80px', width='100%'))
final_box = widgets.VBox([model_init, models_dir, zzz_Args, zzz_Training],layout=widgets.Layout(height='200px', width='100%'))
final_box1 = widgets.VBox([zzz_Extensions, zzz_Upscaler],layout=widgets.Layout(height='80px', width='100%'))
final_box2 = widgets.VBox([zzz_Continue],layout=widgets.Layout(height='90px', width='100%'))

# zzz_Runzzz
def runzzz(b):
    global widgets_box, boton_runzzz
    widgets_box.close()
    final_box.close()
    final_box1.close()
    final_box2.close()
    group_box.close()
    boton_runzzz.close()
    group_title.close()

    if not zzz_Continue.value:
      with capture_output() as cap:
        import os
        if os.path.exists(zzz_widget.value):
          shutil.rmtree(zzz_widget.value)
          import os; link = "https://github.com/A U T O M A T I C 1 1 1 1/s t a b l e-d i f f u s i o n-w e b u i.git"; os.system(f"git clone --depth 1 {link.replace(' ', '')} zzz")

        import os
        if not os.path.exists(zzz_widget.value):
          import os; link = "https://github.com/A U T O M A T I C 1 1 1 1/s t a b l e-d i f f u s i o n-w e b u i.git"; os.system(f"git clone --depth 1 {link.replace(' ', '')} zzz")

        %cd zzz
        !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/zzz_data.txt
        import os; [os.system(line.strip()) for line in open(xyz+'/''zzz_data.txt')];

        clear_output()
      print('[1;96mInstalling S.D.Z ✓')

    if zzz_Continue.value:
      with capture_output() as cap:
        import os
        os.chdir(xyz_widget.value)

        clear_output()
      print('[1;96mPreparing S.D.Z Continue ✓')

    with capture_output() as cap:
      import os
      os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
      !apt --fix-broken install
      !pip install fastapi==0.94.0 -qq

# zzz_Models
      os.chdir(xyz_widget.value)
      %cd $mostaz
      download_command = f"apt-get install -qq -y aria2 && url=\"{Download_Base_Model.value}\" && base_name=$(basename $url) && ((echo $url | grep -q \"drive.google.com\") && gdown --fuzzy $url || ((echo $url | grep -q \"huggingface.co/\") && (hf_token=\"\" && aria2c --console-log-level=error --summary-interval=10 --header=\"Authorization: Bearer $hf_token\" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 $url)"
      os.system(download_command)

# zzz_Extensions
      if zzz_Extensions.value:
        os.chdir(xyz_widget.value)
        %cd models/VAE
        import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/VA E/v ae-f t-m se-8 4 0000-ema-p r uned.ckpt"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        import os; link = "https://huggingface.co/ckpt/s d x l_v ae/resolve/main/s d x l_v a e.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        %cd ..

        if not os.path.exists('Lora'):
          os.makedirs('Lora')
        %cd Lora
        import os; link = "https://civitai.com/api/download/models/6 300 6"; output_file = "Lo wRA.saf etensors"; os.system(f"wget --timestamping {link.replace(' ', '')} -O {output_file.replace(' ', '')}")
        import os; link = "https://civitai.com/api/download/models/117467"; output_file = "polyh edron_sk inny_all.pt"; os.system(f"wget --timestamping {link.replace(' ', '')} -O {output_file.replace(' ', '')}")
        import os; link = "https://civitai.com/api/download/models/117955"; output_file = "topo_v4.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} -O {output_file.replace(' ', '')}")
        import os; link = "https://civitai.com/api/download/models/62833"; output_file = "ad d_det ail.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} -O {output_file.replace(' ', '')}")
        import os; link = "https://civitai.com/api/download/models/118945"; output_file = "epi CRea lismH elper.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} -O {output_file.replace(' ', '')}")
        import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/Lora/epiNoiseoffset_v2.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/Lora/theovercomer8sContrastFix_sd15.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/Lora/to8sHighKeyLORASD15SD2_sd15.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        import os; link = "https://huggingface.co/cas que/detailed_eye-10/resolve/main/deta iled_eye-10.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        #import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/Lora/Clo thi ngAd juster2.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        %cd ../..

        %cd embeddings
        import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/em be ddi ngs/ba d-han ds-5.pt"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        import os; link = "https://huggingface.co/uwg/modelz_base/resolve/main/em be ddi ngs/ea sy neg ative.safetensors"; os.system(f"wget --timestamping {link.replace(' ', '')} ")
        %cd ..

        %cd extensions
        link = "https://github.com/et her ea lx x/ba tc hlinks-w e b u i"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/Co yot e-A/u l t i m ate-ups cale-for-a u t o m a t i c 1 1 1 1"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/Dom inik Doom/a 1 1 1 1-s d-w e b u i-tagcomplete"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/za nl lp/s d-w e b ui-i n f i n i t e-image-browsing"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/c am en du ru/s d-w e b u i-t u n nels"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/yan koo liv eira/s d-w eb ui-p ho to pea-em bed"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/Ph ys ton/s d-w e b ui -p r o m pt-a l l-i n-o ne"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/ric hr obb er2/ca nva s-zo om"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/vi yiv iyi/s t a b l e-d i f f u s i on-w eb u i-zoomimage"; os.system(f"git clone {link.replace(' ', '')}")
        link = "https://github.com/Ar tVe ntu reX/s d-w e b ui-a ge n t-s ch eduler"; os.system(f"git clone {link.replace(' ', '')}")
        #link = "https://github.com/P 2 En j oy/s d-w e b u i-r o op-u n c e ns ored"; os.system(f"git clone {link.replace(' ', '')}")
        #import os; link = "https://github.com/gl uc au ze/s d-w e b u i-f ace s wa p l ab.git"; os.system(f"git clone --depth 1 {link.replace(' ', '')} fwap")
        #import os; actualdir = os.getcwd(); os.chdir(xyz_widget.value+'/extensions/fwap/'); os.system("source venv/bin/activate"); os.system("pip install -r ./requirements.txt"); os.system("pip install -r ./requirements-gpu.txt"); os.chdir(actualdir)
        %cd ..

# zzz_ctrl
      os.chdir(xyz_widget.value)
      %cd extensions
      import os; link = "https://github.com/M i k ub i l l/s d-w e b ui-c o nt r o l n et.git"; os.system(f"git clone --depth 1 {link.replace(' ', '')} ctrl")

      %cd ctrl/models

      if ctrll5.value:
        import os; from concurrent.futures import ThreadPoolExecutor; [executor.submit(os.system, line.strip()) for line in open(xyz_widget.value+'/ctrl15.txt') if not os.path.exists('control_v11f1p_sd15_depth.pth') for executor in [ThreadPoolExecutor(max_workers=len(open(xyz_widget.value+'/ctrl15.txt').readlines()))]];
        %cd ../..

      if ctrlxl.value:
        import os; from concurrent.futures import ThreadPoolExecutor; [executor.submit(os.system, line.strip()) for line in open(xyz_widget.value+'/ctrlxl.txt') if not os.path.exists('control_v11f1p_sd15_depth.pth') for executor in [ThreadPoolExecutor(max_workers=len(open(xyz_widget.value+'/ctrlxl.txt').readlines()))]];
        %cd ../..

# zzz_Upscalers
      if zzz_Upscaler.value:
        os.chdir(xyz_widget.value+'/'+'models')

        if not os.path.exists('ESRGAN'):
            os.makedirs('ESRGAN')
        %cd ESRGAN
        import os; from concurrent.futures import ThreadPoolExecutor; [executor.submit(os.system, line.strip()) for line in open(xyz_widget.value+'/SDZ_Upscalers.txt') if not os.path.exists('4x_UniversalUpscalerV2-Sharper_103000_G.pth') for executor in [ThreadPoolExecutor(max_workers=len(open(xyz_widget.value+'/SDZ_Upscalers.txt').readlines()))]];

# zzz_Training
      selected_option = zzz_Training.value
      if selected_option == "hyako":
        os.chdir(xyz_widget.value)
        %cd extensions
        import os; link = "https://github.com/d d p n 08/k o h y a-s d-s cr ip ts-w e b u i.git"; os.system(f"git clone --depth 1 {link.replace(' ', '')} hyako")
        %cd ..

      elif selected_option == "dramb":
        os.chdir(xyz_widget.value)
        %cd extensions
        import os; link = "https://github.com/d 8 a h a z a r d/s d_d r e a m b o o t h_e x t e n s i on.git"; os.system(f"git clone --depth 1 {link.replace(' ', '')} dramb")
        %cd ..


        clear_output()
    print('[1;90mPreparing S.D.Z System ✓')

    os.chdir(xyz_widget.value)
    selected_option = zzz_Version.value
    if selected_option == "V1.5":
        !python launch.py $zshare --ckpt "$model_init.value" --ckpt-dir "$models_dir.value" $zzz_Args.value

boton_runzzz.on_click(runzzz)
group_title = widgets.HTML(value='')
group_title2 = widgets.HTML(value='')
display(widgets_box, group_title, group_box, group_title2, final_box1, final_box,  final_box2, boton_runzzz)

In [ ]:

import os
import shutil
import urllib.request

from IPython.utils.capture import capture_output
from IPython.display import clear_output
import sys
import fileinput
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact, interactive, fixed
from ipywidgets import Layout, SelectMultiple
from IPython.core.display import HTML

zzz_install = "/content/sample_data"
zzz_install_widget = widgets.Text(value=zzz_install, description="Path install")
zzz_Version = widgets.Dropdown(options=["V1.5", "V2.1"], description="Version")
Download_Base_Model = widgets.Text(value="", description="Link Model")
zzz = "zzz"
zzz_widget = widgets.Text(value=zzz, description="zzz")
xyz = (zzz_install+'/'+'zzz')
xyz_widget = widgets.Text(value=xyz, description="xyz")

model_initp = ""
model_init = widgets.Text(value=model_initp, description="Model_init")
models_dirp = ""
models_dir = widgets.Text(value=models_dirp, description="Models_dir")

zzz_Argsp = ""
zzz_Args = widgets.Text(value=zzz_Argsp, description="Args")
zzz_Training = widgets.Dropdown(options=["", "hyako", "dramb"], description="Training")

ctrll5 = widgets.Checkbox(value=False, description="ctrll5")
ctrlxl = widgets.Checkbox(value=False, description="ctrlxl")


zzz_Extensions = widgets.Checkbox(value=False, description="Extensions")
zzz_Upscaler = widgets.Checkbox(value=False, description="Upscalers")
zzz_Continue = widgets.Checkbox(value=False, description="CONTINUE / / / ")

zshare = "--share --api --xformers --theme=dark"

os.chdir(zzz_install_widget.value)
boton_runzzz = widgets.Button(description="RUN > > >")
boton_runzzz.style.button_color = '#1e1e1e'
boton_runzzz.style.font_weight = 'bold'
boton_runzzz.style.color = 'red'


left_box = widgets.VBox([ctrll5])
right_box = widgets.VBox([ctrlxl])
group_box = widgets.HBox([left_box, right_box],layout=widgets.Layout(height='45px', width='100%'))

left_box.layout.max_width = '150px'
right_box.layout.max_width = '170px'
left_box.layout.overflow = 'hidden'
right_box.layout.overflow = 'hidden'

widgets_box = widgets.VBox([zzz_install_widget, Download_Base_Model],layout=widgets.Layout(height='80px', width='100%'))
final_box = widgets.VBox([model_init, models_dir, zzz_Args, zzz_Training],layout=widgets.Layout(height='200px', width='100%'))
final_box1 = widgets.VBox([zzz_Extensions, zzz_Upscaler],layout=widgets.Layout(height='80px', width='100%'))
final_box2 = widgets.VBox([zzz_Continue],layout=widgets.Layout(height='90px', width='100%'))

# zzz_Runzzz




def runzzz(b):
    global widgets_box, boton_runzzz
    widgets_box.close()
    final_box.close()
    final_box1.close()
    final_box2.close()
    group_box.close()
    boton_runzzz.close()
    group_title.close()

    if not zzz_Continue.value:
      with capture_output() as cap:
        import os
        if os.path.exists(zzz_widget.value):
          shutil.rmtree(zzz_widget.value)
          !git clone --depth 1 https://github.com/wilzamguerrero/vvv zzz

        import os
        if not os.path.exists(zzz_widget.value):
          !git clone --depth 1 https://github.com/wilzamguerrero/vvv zzz

        %cd zzz


        clear_output()
      print('[1;96mInstalling')


    if zzz_Continue.value:
      with capture_output() as cap:
        import os
        os.chdir(xyz_widget.value)

        clear_output()
      print('[1;96mPreparing S.D.Z Continue ✓')

    with capture_output() as cap:
      import os
      os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
      !apt --fix-broken install
      !pip install fastapi==0.94.0 -qq



# zzz_Models
      os.chdir(xyz_widget.value)
      %cd $mostaz
      download_command = f"apt-get install -qq -y aria2 && url=\"{Download_Base_Model.value}\" && base_name=$(basename $url) && ((echo $url | grep -q \"drive.google.com\") && gdown --fuzzy $url || ((echo $url | grep -q \"huggingface.co/\") && (hf_token=\"\" && aria2c --console-log-level=error --summary-interval=10 --header=\"Authorization: Bearer $hf_token\" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 $url)"
      os.system(download_command)



    os.chdir(xyz_widget.value)
    selected_option = zzz_Version.value
    if selected_option == "V1.5":
        !python launch.py $zshare --ckpt "$model_init.value" --ckpt-dir "$models_dir.value" $zzz_Args.value



boton_runzzz.on_click(runzzz)
group_title = widgets.HTML(value='')
group_title2 = widgets.HTML(value='')
display(widgets_box, group_title, group_box, group_title2, final_box1, final_box,  final_box2, boton_runzzz)